# This only uses the cleveland datatset

### Setup

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Check GPU Avaliability

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### Hyper-parameters

In [ ]:
# Hyper-parameters
input_size = 13
hidden_size = 10
num_classes = 5

num_epochs = 100
batch_size = 30
learning_rate = 0.001

### Data

In [ ]:
#pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
#import pandas_profiling as pp
#import warnings
#warnings.filterwarnings('ignore')
#pp.ProfileReport(dataset, title = 'Pandas Profiling report of "dataset"', html = {'style':{'full_width': True}})

In [ ]:
#load and clean dataset
dataset1 = pd.read_csv('/content/drive/My Drive/184A/finalProj/data/processed.cleveland.txt', header = None)
dataset1.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
dataset1 = dataset1.replace('?', np.nan)
dataset1 = dataset1.astype({'age': 'float64','sex': 'category','cp': 'category','trestbps': 'float64','chol': 'float64', 'fbs': 'category', 'restecg': 'category', 'thalach': 'float64', 'exang': 'category', 'oldpeak': 'float64', 'slope': 'category', 'ca': 'category', 'thal': 'category', 'num': 'category'})


In [ ]:
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

np.random.seed(0)
dataset1 = shuffle(dataset1)
sc = StandardScaler()
dataset1.iloc[:,0:-1] = sc.fit_transform(dataset1.iloc[:,0:-1])

def changeTarget(value):
  return 1 if value > 0 else 0
#dataset1['num'] = dataset1['num'].apply(changeTarget) #to make it just 0 and 1's

datasetTrain = dataset1.iloc[0:int(0.75*dataset1.shape[0]), :]
datasetVali = dataset1.iloc[int(0.75*dataset1.shape[0]):, :]

pdXtr = datasetTrain.iloc[:,0:-1]
pdYtr = datasetTrain.iloc[:,-1]
pdXva = datasetVali.iloc[:,0:-1]
pdYva = datasetVali.iloc[:,-1]

npXtr = np.array(pdXtr, dtype='float')
npYtr = np.array(pdYtr, dtype='float')
npXva = np.array(pdXva, dtype='float')
npYva = np.array(pdYva, dtype='float')

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

tensorXtr = torch.Tensor(npXtr) # transform to torch tensor
tensorYtr = torch.Tensor(npYtr)
tensorYtr = tensorYtr.type(torch.LongTensor)
datasetTr = TensorDataset(tensorXtr, tensorYtr) # create your datset
trainloader = DataLoader(datasetTr, batch_size, True) # create your dataloader

tensorXva = torch.Tensor(npXva) # transform to torch tensor
tensorYva = torch.Tensor(npYva)
tensorYva = tensorYva.type(torch.LongTensor)
datasetVa = TensorDataset(tensorXva, tensorYva) # create your datset
valiloader = DataLoader(datasetVa, batch_size, True) # create your dataloader

data, labels = next(iter(trainloader))
print(data.shape, labels.shape)

torch.Size([30, 13]) torch.Size([30])


### Model

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) #input and first hidden layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size) #second hidden layer
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes) #third hidden layer
        #self.relu = nn.ReLU()
        #self.fc4 = nn.Linear(hidden_size, num_classes)  #output

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        #out = self.relu(out)
        #out = self.fc4(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Training

In [ ]:
# Train the model
total_step = len(trainloader)
for epoch in range(num_epochs):
  for i, (data, target) in enumerate(trainloader):
    # Move tensors to the configured device
    data = data.to(device)
    target = target.to(device)

    # Forward pass
    outputs = model(data)
    loss = criterion(outputs, target)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 25 == 0 and (i+1) % 4 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [25/100], Step [4/8], Loss: 0.9997
Epoch [25/100], Step [8/8], Loss: 1.0735
Epoch [50/100], Step [4/8], Loss: 0.9458
Epoch [50/100], Step [8/8], Loss: 0.9359
Epoch [75/100], Step [4/8], Loss: 0.8403
Epoch [75/100], Step [8/8], Loss: 0.7460
Epoch [100/100], Step [4/8], Loss: 0.7040
Epoch [100/100], Step [8/8], Loss: 1.2039


### Test

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for data, labels in valiloader:
        data = data.to(device)
        labels = labels.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        print("Predicted:", predicted, "\nLabel:", labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test daata: {} %'.format(100 * correct / total))

Predicted: tensor([3, 0, 1, 3, 3, 0, 3, 0, 0, 1, 0, 1, 0, 1, 0, 3, 1, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 3, 2, 0, 0]) 
Label: tensor([0, 1, 2, 3, 3, 0, 4, 0, 0, 2, 0, 3, 2, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 0,
        2, 0, 4, 0, 2, 0])
Predicted: tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0,
        3, 3, 0, 0, 3, 0]) 
Label: tensor([0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 3, 1, 1, 0, 0, 0, 0,
        3, 2, 0, 0, 2, 2])
Predicted: tensor([0, 3, 0, 0, 3, 0, 3, 0, 0, 2, 3, 0, 0, 0, 3]) 
Label: tensor([0, 3, 1, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 4])
Accuracy of the network on the test daata: 64.0 %


In [ ]:
#with open('/content/drive/My Drive/184A/finalProj/data/processed.cleveland.txt', 'r') as f:
#  f.write('Hello Google Drive!')
#!cat /content/drive/My\ Drive/184A/finalProj/data/processed.cleveland.txt